In [2]:
cd ~/bench

/home/soda/rcappuzz/work/benchmark-join-suggestions


This notebook is used to combine all results from various runs in to a single file for simplicity and better storage. 

In [3]:
import polars as pl
from pathlib import Path
from src.utils.logging import read_and_process, read_logs

In [4]:
cfg = pl.Config()
cfg.set_fmt_str_lengths(150)

polars.config.Config

In [5]:
dest_path = Path("results/overall")
overall_list = []

`results/logs/0363-mp1a3by4`
- Full run
- All tables
- Wordnet full
- Aggregation first 

In [6]:
r_path = "results/logs/0363-mp1a3by4"
df_raw = read_logs(exp_name=None, exp_path=r_path)
overall_list.append(df_raw)
df_raw.write_parquet(Path(dest_path, "wordnet_general_first.parquet"))

### Open Data

```
"results/logs/0373-csmtrlcd",
"results/logs/0374-7226gvfk",
"results/logs/0375-mjbya9c5",
"results/logs/0376-c8xw7pry",
"results/logs/0377-q83k1stf"
```

`results/logs/0375-mjbya9c5` is a duplicate of 0374

In [8]:
paths = [
    "results/logs/0372-35kmeaz3",
    "results/logs/0373-csmtrlcd",
    "results/logs/0374-7226gvfk",
    "results/logs/0376-c8xw7pry",
    "results/logs/0377-q83k1stf",
    "results/logs/0381-roaef1ce",
]

open_data_list = []

for path in paths:
    df_raw = read_logs(exp_name=None, exp_path=path)
    open_data_list.append(df_raw)

df_concat = pl.concat(open_data_list)
df_concat.write_parquet(Path(dest_path, "open_data_general_first.parquet"))
overall_list.append(df_concat)

In [9]:
df_concat.select(pl.col("base_table").unique()).to_series().to_list()

['us_accidents-depleted_County-open_data',
 'housing_prices-depleted_County-open_data',
 'company_employees-depleted_name-open_data',
 'us_elections-depleted_county_name-open_data',
 'movies_vote-depleted_title-open_data',
 'movies-depleted_title-open_data']

In [10]:
df_overall = pl.concat(overall_list)
df_overall.write_parquet(Path(dest_path, "wordnet_open_data_first.parquet"))

`results/logs/0382-bfft0brr`
- Only best single join and highest containment
- All tables
- Wordnet full
- DFS 

In [35]:
r_path = "results/logs/0382-bfft0brr"
df_raw = read_logs(exp_name=None, exp_path=r_path)
# overall_list.append(df_raw)
df_raw.write_parquet(Path(dest_path, "wordnet_dfs.parquet"))
df_base = pl.read_parquet(Path(dest_path, "wordnet_general_first.parquet"))
drop = ["scenario_id", "status", "target_dl", "rmse", "f1score", "auc", "n_cols","budget_type", "budget_amount", "epsilon"]
df_c = pl.concat([df_base.drop(drop), df_raw.drop(drop)])
f = {"chosen_model": "linear", "jd_method": "exact_matching"}
df_aggr = df_c.filter(**f).filter(pl.col("estimator").is_in(["highest_containment", "best_single_join"]) )
df_aggr.write_parquet(Path(dest_path, "wordnet_aggr.parquet"))